# Appendix A

### A.1 Importing Libraries

In [1]:
import tweepy
import pandas as pd
from tweepy import Cursor
import pytz
import numpy as np

### A.2 Tweepy set up

In [2]:
#my keys should put them in another file and hide them before putting this on git
API_key = "7M1xKUhbvU4NITzZAoeeuSBZb"
API_secret_key = "ksSxd57SkrL6kzg1PM5uZ0TDLvUz6u7078Im2KqL8bBKpxc168"
access_token = "705069617295790080-2fmdiurkwA4MrclmfrcXbCZOPmwspEc"
access_token_secret = "mAYVj092P1KLTyZUjuOm2cBWGgy1OAYQYrFZhOFmW2K5N"

In [3]:
# Setting up the tweet extraction
auth  = tweepy.OAuthHandler(API_key,API_secret_key)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)
weightroom_id  = "WesternWeightRm"

In [4]:
#specifying weight room twitter id as user to be extracted
rec_user  = api.get_user(weightroom_id)
timeline = rec_user.timeline()

### A.2 Tweet Retrival

In [5]:
#Using tweepy interfact to fetch tweets 
#using build in interfact to format dates and tweets into python lists 
tweets = Cursor(api.user_timeline, id=weightroom_id).items()
dates = []
text = []
for tweet in tweets:
        dates.append(tweet.created_at)
        text.append(tweet.text)

In [6]:
#Function deEmojify 
#input 
#InputString 
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

In [7]:
#Iterating through tweet and date list to make a pandas dataframe
#Uses demojify  to remove emojis from tweets 
#Uses regular expresions to find only number values from tweets then storing them into numbers_wr and numbers_cm lists
#creates final dataframe and makes a list of not used tweets 
import string
import re
data = pd.DataFrame({'date' : dates, 'tweets' : text})
no_emoji= []
numbers_wr = []
numbers_cm = []
u = []
not_used = []
for i in range(len(data.iloc[:,1]) ):
    emoji_free  = deEmojify(data.iloc[i,1])
    emoji_free = emoji_free.replace("&amp;","")
    emoji_free = emoji_free.replace("\n" , "")
    values  = re.findall(r'(\w*\d+)',emoji_free)
  
    if(len(values) == 2):
        numbers_wr.append(values[0])
        numbers_cm.append(values[1])

    else:
        not_used.append(data.iloc[i,1])
    #if(nums == None ):
     #   u.append(emoji_free)
  
        

### A.3 Data Exportation

In [8]:
#Dropping tweets to not use and making CM and WR value columns using values obtained before
to_drop = data[data.tweets.isin( not_used)].index
clean_data = data.drop(to_drop)
clean_data['CM'] =  numbers_cm
clean_data['WR'] =  numbers_wr

In [10]:
clean_data.head()

,date,tweets,CM,WR
0,2020-03-15 20:37:58,WR 47 CM 12,12,47
1,2020-03-15 17:55:33,WR 52 CM 22 🎾,22,52
2,2020-03-15 16:34:29,WR 48 CM 24 😀,24,48
3,2020-03-15 15:55:35,WR 23 CM 13,13,23
4,2020-03-15 15:23:41,WR 19\nCM 6 🏃🏾‍♀️😊👍🏼🌬😷,6,19


In [9]:
clean_data.to_csv("clean_tweeets.csv" )